# 01背包问题
> [P1048 采药](https://www.luogu.com.cn/problem/P1048)

**题目描述（节选）**

医师为了判断他的资质，给他出了一个难题。医师把他带到一个到处都是草药的山洞里对他说：“孩子，这个山洞里有一些不同的草药，采每一株都需要一些时间，每一株也有它自身的价值。我会给你一段时间，在这段时间里，你可以采到一些草药。如果你是一个聪明的孩子，你应该可以让采到的草药的总价值最大。”
如果你是辰辰，你能完成这个任务吗？

**【数据范围】**

- 对于 $30\%$ 的数据，$M \le 10$；
- 对于全部的数据，$M \le 100$，$T \le 1000$

定义数组 $cost_i$, $value_i$ 分别用于存储第 $i$ 株草药的采集时间和价值：

In [1]:
T, M = map(int, input().split())

cost = list()
value = list()

for i in range(M):
    c, v = map(int, input().split())
    cost.append(c)
    value.append(v)

 71 3
 72 1000
 70 1
 1 2



对于本题，我们可以定义一个二维数组 $DP_{i,\ j}$ 来表示采集前 $i$ 株草药、总共拥有 $j$ 的时间的时候，能采到的草药价值最大值

则答案即为 $DP_{M-1, T}$

那么对于第一株草药（下标为 $0$）来讲：
- 如果剩余的时间不足以采集这株草药，那么总价值只能是 $0$
- 如果剩余的时间足够采集这株草药，由于仅剩这一株草药可以采，则必然要采集，此时总价值即为 $value_0$

代码如下：

In [2]:
dp = [[0 for i in range(T+1)] for j in range(M)]

for i in range(T + 1):
    if i < cost[0]:
        dp[0][i] = 0
    else:
        dp[0][i] = value[i]

接下来，我们遍历 $i$ 从第二株草药到最后一株草药，对于每一种剩余时间 $j$：
- 如果剩余的时间不足以采集当前这株草药，那么就直接从前一株开始采，最后总价值也就是用同样的时间从前一株开始采的总价值
    即 $DP_{i-1, j}$
- 如果剩余的时间足够采集当前这株草药，那么由两种可能：
    - 不采集这株草药，总价值就是用同样的时间从前一株开始采的总价值
        即 $DP_{i-1, j}$
    - 采集这株药，那么采集完成后可以用剩余的时间从前一株开始采，总价值是它们的和
        即 $DP_{i-1, j-cost_i}+value_i$
        
    最大总价值会从这两种产生

In [3]:
for i in range(1, M):
    for j in range(T + 1):
        if j < cost[i]:
            dp[i][j] = dp[i-1][j]
        else:
            dp[i][j] = max(dp[i-1][j], value[i] + dp[i-1][j-cost[i]])

输出答案

In [4]:
print(dp[M-1][T])

3


以上代码可以正常运行，但有个问题—— $DP$ 数组可能会非常大

所以我们准备修改 $DP$ 数组，使它的空间减小

我们可以发现，每一次修改 $DP_{i, j}$ 的值时，我们只会使用到数组 $DP_{i-1}$ 中的值，且第二下标的值不小于 $j$

所以我们完全不需要存储除 $i-1$ 这一行以外的其他数据

将 $DP$ 改为一维数组，其每次存储的值都是上次修改后的值，为原 $i-1$ 行的值

**这里需要注意：由于其下标不小于当前下标，所以须从后向前遍历，才不会使用刚刚修改过的、原 $i$ 行的值**

修改代码如下：

In [5]:
dp = [0 for i in range(T+1)]

for i in range(T + 1):
    if i < cost[0]:   # 请注意这里，i 只有从 cost[0] 开始才会修改dp的值，遍历前 cost[0] 个值没有必要
        dp[i] = 0
    else:
        dp[i] = value[i]
        
for i in range(1, M):
    for j in range(T, 0, -1):  # j=0 时一定无可采集，0即可
        if j < cost[i]:
            dp[j] = dp[j]  # 这是一句没有用的语句
        else:
            dp[j] = max(dp[j], value[i] + dp[j-cost[i]])

我在上面的代码中标注了几处可以删除的语句，现在我们把它修改成这样：

In [6]:
dp = [0 for i in range(T+1)]

for i in range(cost[0], T + 1):
    dp[i] = value[0]
        
for i in range(1, M):
    for j in range(T, cost[i] - 1, -1):
        dp[j] = max(dp[j], value[i] + dp[j-cost[i]])

别忘了输出：

In [7]:
print(dp[T])

3


最终代码:

In [9]:
T, M = map(int, input().split())

cost = list()
value = list()

for i in range(M):
    c, v = map(int, input().split())
    cost.append(c)
    value.append(v)
    
dp = [0 for i in range(T+1)]

for i in range(cost[0], T + 1):
    dp[i] = value[0]
        
for i in range(1, M):
    for j in range(T, cost[i] - 1, -1):
        dp[j] = max(dp[j], value[i] + dp[j-cost[i]])
        
print(dp[T])

 70 3
 71 100
 1 1
 69 2


3


![这是什么？](https://cdn.luogu.com.cn/upload/image_hosting/ar0mwgv8.png)

> 话说 Python 的内存占用时真的大